In [1]:
cd ..

/home/tommy/Scrivania/Polimi/Honours Programme/SCFR


In [2]:
from data_structures.cfr_trees import *
from data_structures.trees import *
from cfr_code.reconstruction_cfr import *
from cfr_code.sample_cfr import *
from utilities.graphs import *
from games.kuhn import build_kuhn_tree
from games.leduc import build_leduc_tree
from games.goofspiel import build_goofspiel_tree, TieSolver

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
def test_random(d, p, i, RCFR_it, RCFR_rec_every, SCFR_it, xaxis, ybottom = 0, reconstructPlayersTogether = False,
                show_perc = False):
    tree = randomTree(d, p, i)
    cfr_tree = CFRTree(tree)
    ct1 = CFRTree(tree)
    res1 = SolveWithReconstructionCFR(ct1, RCFR_it, show_perc = show_perc, checkEveryIteration = 1, 
                                      reconstructEveryIteration = RCFR_rec_every,
                                      reconstructPlayersTogether = reconstructPlayersTogether,
                                      reconstructWithOptimalPlan = False)
    
    #ct3 = CFRTree(tree)
    #res3 = SolveWithReconstructionCFR(ct3, RCFR_it, show_perc = show_perc, checkEveryIteration = 1, 
    #                                  reconstructEveryIteration = RCFR_rec_every,
    #                                  reconstructPlayersTogether = reconstructPlayersTogether,
    #                                  reconstructWithOptimalPlan = False)
    ct2 = CFRTree(tree)
    res2 = SolveWithSampleCFR(ct2, SCFR_it, show_perc = show_perc, checkEveryIteration = 1)
    comparative_epsilon_graph([res1, res2], xaxis = xaxis, ybottom = ybottom)
    return (res1, res2, tree, ct1, ct2)

In [ ]:
def test_kuhn():
    tree = build_kuhn_tree(3, 6)
    cfr_tree = CFRTree(tree)
    print("------------ RCFR -------------")
    ct1 = CFRTree(tree)
    res1 = SolveWithReconstructionCFR(ct1, 100, show_perc = True, checkEveryIteration = 1, 
                                      reconstructEveryIteration = 10)
    print("------------ RCFR+ -------------")
    ct2 = CFRTree(tree)
    res2 = SolveWithReconstructionCFR(ct2, 100, show_perc = True, checkEveryIteration = 1, 
                                      reconstructEveryIteration = 10, use_cfr_plus = True)
    print("------------ SCFR -------------")
    ct3 = CFRTree(tree)
    res3 = SolveWithSampleCFR(ct3, 100, show_perc = True, checkEveryIteration = 1)
    comparative_epsilon_graph([res1, res2, res3], xaxis = 'time')
    return (res1, res2, res3, tree)


(res1, res2, res3, tree) = test_kuhn()
    
print(res1["tot_time"])
print(res2["tot_time"])
print(res3["tot_time"])

# Multiple players at once

In [ ]:
test_random(8, 3, 1, 100, 1, 100, 'iterations')

In [ ]:
ct1.root.buildRealizationForm(None, 1)

In [ ]:
leaves = set()
ct1.root.find_terminals(leaves)

In [ ]:
for l in leaves:
    print(l.omega)

In [ ]:
(res1, res2, tree) = test_random(3, 2, 1, 100, 1, 300, 'time', ybottom = -0.5, reconstructPlayersTogether = True)

In [ ]:
tree.display()

In [ ]:
res1['joint'].plans

In [ ]:
res1['graph_data'][-1:]

In [ ]:
res2['joint'].plans

In [ ]:
c_tree = CFRTree(tree)
rrr = SolveWithReconstructionCFR(c_tree, 100, reconstructPlayersTogether = True)
c_tree2 = CFRTree(tree)
rrr2 = SolveWithSampleCFR(c_tree2, 100)

In [ ]:
for iset in c_tree.information_sets.values():
    print(str(iset) + " - " + str(iset.current_strategy))
    
print("----------------------")
    
for iset in c_tree2.information_sets.values():
    print(str(iset) + " - " + str(iset.current_strategy))

# Profiling

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f CFRInformationSet.updateSupportingPlan test_random(6, 3, 0, 100, 1, 100, xaxis = 'time', show_perc = True)

# Do not optimize over plans

In [ ]:
dims = [(6, 2, 1),(8, 2, 1),(10, 2, 1),(12, 2, 1),(13, 2, 1),
        (6, 2, 0),(8, 2, 0),(10, 2, 0),(12, 2, 0),(13, 2, 0)]

for (d, p, i) in dims:
    print("----------------" + str((d, p, i)) + "----------------")
    _ = test_random(d, p, i, 100, 1, 200, xaxis = 'time')

In [ ]:
dims = [(5, 3, 1),(6, 3, 1),(7, 3, 1),(8, 3, 1),(9, 3, 1),
       (5, 3, 0),(6, 3, 0),(7, 3, 0),(8, 3, 0),(9, 3, 0)]

for (d, p, i) in dims:
    print("----------------" + str((d, p, i)) + "----------------")
    _ = test_random(d, p, i, 100, 1, 500, xaxis = 'time')

# Leduc

In [ ]:
def test_leduc(players, rank, show_perc = False, iterations = (100, 100, 200), reconstructEveryIteration = 1):
    tree = build_kuhn_tree(players, rank)
    cfr_tree = CFRTree(tree)
    print("------------ RCFR -------------")
    ct1 = CFRTree(tree)
    res1 = SolveWithReconstructionCFR(ct1, iterations[0], show_perc = show_perc, checkEveryIteration = 1, 
                                      reconstructEveryIteration = reconstructEveryIteration)
    print("------------ RCFR_nopt -------------")
    ct2 = CFRTree(tree)
    res2 = SolveWithReconstructionCFR(ct2, iterations[1], show_perc = show_perc, checkEveryIteration = 1, 
                                      reconstructEveryIteration = reconstructEveryIteration, 
                                      reconstructWithOptimalPlan = False)
    print("------------ SCFR -------------")
    ct3 = CFRTree(tree)
    res3 = SolveWithSampleCFR(ct3, iterations[2], show_perc = show_perc, checkEveryIteration = 1)
    comparative_epsilon_graph([res1, res2, res3], xaxis = 'time', legend = ['RCFR','RCFR_nopt', 'SCFR'])
    comparative_epsilon_graph([res1, res2, res3], xaxis = 'time', legend = ['RCFR','RCFR_nopt'], ylims = (0, 0.1))
    return (res1, res2, res3, tree)

In [ ]:
_ = test_leduc(3, 5, iterations = (100, 400, 500), show_perc = True)

# Goofspiel

In [5]:
def test_goofspiel(players, rank, show_perc = False, iterations = (100, 100, 200), reconstructEveryIteration = 1):
    tree = build_goofspiel_tree(players, rank, tie_solver = TieSolver.Accumulate)
    cfr_tree = CFRTree(tree)
    print("------------ SCFR -------------")
    ct3 = CFRTree(tree)
    res3 = SolveWithSampleCFR(ct3, iterations[2], show_perc = show_perc, checkEveryIteration = 1)
    print("------------ RCFR -------------")
    #ct1 = CFRTree(tree)
    #res1 = SolveWithReconstructionCFR(ct1, iterations[0], show_perc = show_perc, checkEveryIteration = 1, 
    #                                  reconstructEveryIteration = reconstructEveryIteration)
    print("------------ RCFR_nopt -------------")
    ct2 = CFRTree(tree)
    res2 = SolveWithReconstructionCFR(ct2, iterations[1], show_perc = show_perc, checkEveryIteration = 1, 
                                      reconstructEveryIteration = reconstructEveryIteration, 
                                      reconstructWithOptimalPlan = False)
    comparative_epsilon_graph([res1, res2, res3], xaxis = 'time', legend = ['RCFR','RCFR_nopt', 'SCFR'])
    comparative_epsilon_graph([res1, res2, res3], xaxis = 'time', legend = ['RCFR','RCFR_nopt'], ylims = (0, 0.1))
    return (res1, res2, res3, tree)

In [ ]:
_ = _ = test_goofspiel(3, 4, iterations = (10, 10, 10), show_perc = True)

------------ SCFR -------------
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
100.0%
110.0%
------------ RCFR -------------
------------ RCFR_nopt -------------
